<a href="https://colab.research.google.com/github/kannandreams/AI-CodeBook/blob/master/nlp/word_embedding/imdb_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow as tf
print(tf.__version__)
tf.enable_eager_execution()


1.15.0


In [2]:
import tensorflow_datasets as tfds
imdb,info = tfds.load('imdb_reviews',with_info=True,as_supervised=True)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQMGACS/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQMGACS/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQMGACS/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
import numpy as np

train_data,test_data = imdb['train'],imdb['test']
train_sentences=[]
test_sentences=[]

train_labels=[]
test_labels=[]

for s,l in train_data:
  train_sentences.append(str(s.numpy()))
  train_labels.append(l.numpy())

for s,l in test_data:
  test_sentences.append(str(s.numpy()))
  test_labels.append(l.numpy())



In [0]:
#sentence encoding
vocab_size = 10000 
embedding_dim = 16
max_length = 120 # review length
trunc_type='post'
oov_token="<oov"

# Steps : Tokenize -> create word index -> convert to sequences -> do padding for same size -> testing sequences should be tokenized based on training word index

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(train_sentences)
padded = pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences,maxlen=max_length)

In [34]:
#define  the NN
model = tf.keras.Sequential([
              tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(6,activation='relu'),
              tf.keras.layers.Dense(1,activation='sigmoid') # two classes , so one neuron
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.fit(padded,
          np.array(train_labels),
          epochs=10,
          validation_data=(test_padded,np.array(test_labels)),
          verbose=2
)

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 - 5s - loss: 0.4880 - acc: 0.7504 - val_loss: 0.3510 - val_acc: 0.8437
Epoch 2/10
25000/25000 - 5s - loss: 0.2432 - acc: 0.9094 - val_loss: 0.3621 - val_acc: 0.8408
Epoch 3/10
25000/25000 - 5s - loss: 0.1032 - acc: 0.9732 - val_loss: 0.4328 - val_acc: 0.8323
Epoch 4/10
25000/25000 - 5s - loss: 0.0301 - acc: 0.9955 - val_loss: 0.5604 - val_acc: 0.8196
Epoch 5/10
25000/25000 - 5s - loss: 0.0115 - acc: 0.9987 - val_loss: 0.5846 - val_acc: 0.8272
Epoch 6/10
25000/25000 - 5s - loss: 0.0049 - acc: 0.9994 - val_loss: 0.6346 - val_acc: 0.8242
Epoch 7/10
25000/25000 - 5s - loss: 0.0015 - acc: 0.9999 - val_loss: 0.6816 - val_acc: 0.8260
Epoch 8/10
25000/25000 - 5s - loss: 5.6029e-04 - acc: 1.0000 - val_loss: 0.7273 - val_acc: 0.8276
Epoch 9/10
25000/25000 - 5s - loss: 3.0875e-04 - acc: 1.0000 - val_loss: 0.7577 - val_acc: 0.8275
Epoch 10/10
25000/25000 - 5s - loss: 1.7767e-04 - acc: 1.0000 - val_loss: 0.7975 - val_acc: 0.82

In [43]:
sample_review = ["Good Movie!"]
val_seq = tokenizer.texts_to_sequences(sample_review)
val_padded = pad_sequences(val_seq,maxlen=max_length,truncating=trunc_type)
print(val_seq)
print(val_padded)
model.predict(val_padded)

[[52, 18]]
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 52 18]]


array([[0.6771755]], dtype=float32)

**To view this in tensorflow embedding projector**

In [50]:
weights = model.layers[0].get_weights()[0]
print(weights.shape)
print(weights)

(10000, 16)
[[ 2.3717916e-02 -4.1335560e-02  4.8740409e-02 ... -5.4761455e-03
   2.5576169e-02 -2.4499504e-02]
 [ 5.5061463e-02 -7.9436496e-02  6.8881005e-02 ... -5.3180512e-03
  -1.2371118e-02 -7.7541757e-05]
 [ 3.9728680e-03 -5.2733060e-02  9.4237372e-02 ...  2.5939497e-03
   6.0456324e-02 -3.9922852e-02]
 ...
 [-1.6018631e-01 -9.2584811e-02 -3.5184532e-02 ...  1.1890968e-01
   9.6716762e-02  2.3366053e-02]
 [-2.5773803e-02 -2.8773749e-02  6.6927269e-02 ... -4.8390757e-02
   2.8916422e-02  2.7901145e-02]
 [ 3.5977900e-02 -1.1615841e-01 -8.4580844e-03 ... -7.0351437e-02
  -5.8211260e-02  6.4553164e-02]]


In [0]:
# generate vector and meta auto files to upload in embedding projector and visualize


import io

reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

vectors = io.open("imdb_vectors.tsv",'w',encoding='utf-8')
meta = io.open("imdb_meta.tsv",'w',encoding='utf-8')

for word_num in range(1,vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  meta.write(word + "\n")
  vectors.write('\t'.join([str(w) for w in embeddings]) + "\n")

vectors.close()
meta.close()

In [0]:
# download the file

try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download("imdb_vectors.tsv")
  files.download("imdb_meta.tsv")

Go to https://projector.tensorflow.org/

1. Load the vector and meta files
2. visualize it.